### 一、unsloth快速使用入门

#### 1.借助unsloth进行模型推理

In [2]:
# modelscope download --model deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --local_dir DeepSeek-R1-Distill-Qwen-7B
# modelscope download --model deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B --local_dir DeepSeek-R1-Distill-Qwen-1.5B

In [ ]:
# from huggingface_hub import snapshot_download
#
# # 下载模型到本地
# snapshot_download(repo_id="unsloth/DeepSeek-R1-Distill-Qwen-1.5B", local_dir="unsloth-DeepSeek-R1-Distill-Qwen-1.5B")

In [1]:
from unsloth import FastLanguageModel
# import os
# os.environ['CUDA_PATH'] = r'C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.4'

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


}- 尝试用unsloth进行LLama模型推理

&emsp;&emsp;首先设置关键参数，并读取模型：

In [2]:
max_seq_length = 2048 
dtype = None 
load_in_4bit = False
# load_in_4bit = True

> 注，若显存不足，则可以load_in_4bit = True，运行4 bit量化版。

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "../DeepSeek-R1-Distill-Qwen-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.2.12: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4080 SUPER. Max memory: 15.992 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


E:\miniconda3\envs\llama_factory\Lib\site-packages\unsloth\models\llama.py:1255: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  self.register_buffer("cos_cached", emb.cos().to(dtype=dtype, device=device, non_blocking=True), persistent=False)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

../DeepSeek-R1-Distill-Qwen-7B does not have a padding token! Will use pad_token = <|vision_pad|>.


> 在INT4量化情况下，8B模型推理仅需7G左右显存。

此时model就是读取进来的DeepSeek R1 8B蒸馏模型：

In [4]:
model.to("cuda")

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
 

In [5]:
# model

而tokenizer则是分词器：

In [7]:
# tokenizer

将模型调整为推理模式：

In [10]:
FastLanguageModel.for_inference(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
 

然后即可和模型进行对话：

In [6]:
question = "请介绍你自己？包括兴趣、特长、工作经历等。大概200字左右。"

然后这里我们首先需要借助分词器，将输入的问题转化为标记索引：

In [7]:
inputs = tokenizer([question], return_tensors="pt").to("cuda")

In [8]:
inputs

{'input_ids': tensor([[151646,  14880, 100157, 107828,  11319, 100630, 100565,   5373, 112526,
           5373,  99257, 100798,  49567,   1773, 102201,     17,     15,     15,
          18600, 101081,   1773]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

最后再带入inputs进行对话

In [8]:
# 调整生成参数
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=500,  # 限制生成的长度
    attention_mask=inputs.attention_mask,
    temperature=0.7,  # 控制随机性
    use_cache=True,
)

此时得到的回复也是词索引：

In [13]:
# outputs

同样需要分词器将其转化为文本：

In [9]:
response = tokenizer.batch_decode(outputs,skip_special_tokens=True)

In [10]:
response

['请介绍你自己？包括兴趣、特长、工作经历等。大概200字左右。看起来挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺挺']

In [16]:
print(response[0])

请介绍你自己？包括兴趣、特长、工作经历等。大概200字左右。））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））））


至此我们就完成了unsloth模型推理流程。

- 尝试使用unsloth调用Qwen模型

&emsp;&emsp;类似的，我们也可以使用unsloth调用Qwen蒸馏模型

In [1]:
model_qwen, tokenizer_qwen = FastLanguageModel.from_pretrained(
    model_name = "../DeepSeek-R1-Distill-Qwen-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

NameError: name 'FastLanguageModel' is not defined

In [12]:
FastLanguageModel.for_inference(model_qwen) 

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
 

In [13]:
inputs = tokenizer_qwen([question], return_tensors="pt").to("cuda")

outputs = model_qwen.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

response = tokenizer_qwen.batch_decode(outputs)

print(response[0])

<｜begin▁of▁sentence｜>请问如何证明根号2是无理数？”””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””””

> 注，以下实验均以DeepSeek R1 LLama3蒸馏模型为例进行演示和讲解，若想替换为DeepSeek R1 Qwen模型，则可以直接替换模型名称即可。

#### 2.带入问答模板进行回答

- 结构化输入方法

In [8]:
prompt_style_chat = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>{}"""

In [25]:
prompt_style_chat = """请写出一个恰当的回答来完成当前对话任务。

### Instruction:
你是一名助人为乐的助手。

### Question:
{}

### Response:
<think>{}"""

In [9]:
question = "一个患有急性阑尾炎的病人已经发病5天了，腹痛少有减轻但任然发热，在体检时发现右下腹有压痛的包块，此时应该如何处理？"

In [26]:
[prompt_style_chat.format(question, "")]

['请写出一个恰当的回答来完成当前对话任务。\n\n### Instruction:\n你是一名助人为乐的助手。\n\n### Question:\n请证明根号2是无理数。\n\n### Response:\n<think>']

In [11]:
inputs = tokenizer([prompt_style_chat.format(question, "")], return_tensors="pt").to("cuda")

In [12]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=1200,
    use_cache=True,
    attention_mask=inputs.attention_mask,
)

In [13]:
response = tokenizer.batch_decode(outputs)

In [14]:
response

['<｜begin▁of▁sentence｜>Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\n一个患有急性阑尾炎的病人已经发病5天了，腹痛少有减轻但任然发热，在体检时发现右下腹有压痛的包块，此时应该如何处理？\n\n### Response:\n<think>\n\n要回答这个问题，需要了解病人的情况和诊断程序。\n\n诊断过程包括：\n1. 病人正在服用药物和进行诊断。\n2. 检查身体指标，如血压、 pulse、和现在的情况。\n3. 检查身体是否有异常，如感染、炎症、或药物影响。\n4. 考虑药物和诊断程序的综合影响。\n\n诊断步骤：\n- 评估药物影响和诊断程序。\n- 考虑综合影响和诊断结果。\n- 检查身体指标，如血压、 pulse、和现在的情况。\n\n因此，诊断程序包括：\n1. 评估药物影响和诊断程序的综合影响。\n2. 考虑综合影响和诊断结果。\n3. 检查身体指标，如血压、 pulse、和现在的情况。\n\n最终，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。\n\n因此，诊断程序包括评估药物影响和诊断步骤的综合影响，以及身体指标的检查。\n\n最终，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的

In [18]:
print(len(response[0].split("### Response:")[1]))

2084


In [15]:
print(response[0].split("### Response:")[1])


<think>

要回答这个问题，需要了解病人的情况和诊断程序。

诊断过程包括：
1. 病人正在服用药物和进行诊断。
2. 检查身体指标，如血压、 pulse、和现在的情况。
3. 检查身体是否有异常，如感染、炎症、或药物影响。
4. 考虑药物和诊断程序的综合影响。

诊断步骤：
- 评估药物影响和诊断程序。
- 考虑综合影响和诊断结果。
- 检查身体指标，如血压、 pulse、和现在的情况。

因此，诊断程序包括：
1. 评估药物影响和诊断程序的综合影响。
2. 考虑综合影响和诊断结果。
3. 检查身体指标，如血压、 pulse、和现在的情况。

最终，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序包括评估药物影响和诊断步骤的综合影响，以及身体指标的检查。

最终，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序包括评估药物影响和诊断步骤的综合影响，以及身体指标的检查。

最终，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序包括评估药物影响和诊断步骤的综合影响，以及身体指标的检查。

最终，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序是药物影响和诊断步骤的综合影响，以及身体指标的检查。

因此，诊断程序是

- 复杂问题测试

In [27]:
question = "请证明根号2是无理数。"

In [28]:
inputs = tokenizer([prompt_style_chat.format(question, "")], return_tensors="pt").to("cuda")

In [29]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=1200,
    use_cache=True,
    attention_mask=inputs.attention_mask,
)

In [30]:
response = tokenizer.batch_decode(outputs)

In [32]:
print(response[0].split("### Response:")[1])

['<｜begin▁of▁sentence｜>请写出一个恰当的回答来完成当前对话任务。\n\n### Instruction:\n你是一名助人为乐的助手。\n\n### Question:\n请证明根号2是无理数。\n\n### Response:\n<think>\n\n</think>\n\n您的请求已经得到满足。\n\n### Command\n\nCommand\n\n### Question\n\nQuestion\n\n### Question\n\nQuestion\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n### Question\n\n##

#### 3.原始模型的医疗问题问答

- 重新设置问答模板

In [21]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>{}"""

翻译如下：

```python
prompt_style = """以下是一个任务说明，配有提供更多背景信息的输入。
请写出一个恰当的回答来完成该任务。
在回答之前，请仔细思考问题，并按步骤进行推理，确保回答逻辑清晰且准确。

### Instruction:
您是一位具有高级临床推理、诊断和治疗规划知识的医学专家。
请回答以下医学问题。
```

&emsp;&emsp;接下来我们抽取部分medical-o1-reasoning-SFT数据集中问题进行提问，并查看初始状态下模型回答结果。

In [22]:
question_1 = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"

翻译：一位61岁的女性，有长期在咳嗽或打喷嚏等活动中发生不自主尿液流失的病史，但夜间没有漏尿。她接受了妇科检查和Q-tip测试。根据这些检查结果，膀胱测量（cystometry）最可能会显示她的残余尿量和逼尿肌收缩情况如何？









In [7]:
question_2 = "Given a patient who experiences sudden-onset chest pain radiating to the neck and left arm, with a past medical history of hypercholesterolemia and coronary artery disease, elevated troponin I levels, and tachycardia, what is the most likely coronary artery involved based on this presentation?"

翻译：面对一位突发胸痛并放射至颈部和左臂的患者，其既往病史包括高胆固醇血症和冠状动脉疾病，同时伴有升高的肌钙蛋白I水平和心动过速，根据这些临床表现，最可能受累的冠状动脉是哪一条？

- 问答测试

In [11]:
inputs1 = tokenizer([prompt_style.format(question_1, "")], return_tensors="pt").to("cuda")


outputs1 = model.generate(
    input_ids=inputs1.input_ids,
    max_new_tokens=1200,
    use_cache=True,
    attention_mask=inputs1.attention_mask,
)

response1 = tokenizer.batch_decode(outputs1)

KeyboardInterrupt: 

In [17]:
response1

["<｜begin▁of▁sentence｜>Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nPlease answer the following medical question. \n\n### Question:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Response:\n<think>\n\n### Answer:\n\nThe 6-year-old woman is the patient. The patient's findings are examined. Based on the test results, what would the cyst most reveal a

In [19]:
print(response1[0].split("### Response:")[1])


<think>

### Answer:

The 6-year-old woman is the patient. The patient's findings are examined. Based on the test results, what would the cyst most reveal about residual volume and andrus counter?

The patient's test results are considered. Based on the diagnosis, what would the cyst most reveal about residual volume and andrus counter?

The patient's test results are used to determine the diagnosis. Based on the diagnosis, what would the cyst most reveal about residual volume and andrus counter?

The patient's test results are used to determine the diagnosis. Based on the diagnosis, what would the cyst most reveal about residual volume and andrus counter?

The patient's test results are used to determine the diagnosis. Based on the diagnosis, what would the cyst most reveal about residual volume and andrus counter?

The patient's test results are used to determine the diagnosis. Based on the diagnosis, what would the cyst most reveal about residual volume and andrus counter?

The pat

翻译如下：

\<think>  
好的，我正在尝试分析这个医学问题。我们一步步来分解。患者是一位61岁的女性，有在咳嗽或打喷嚏等活动中发生不自主尿液流失的病史，但她夜间没有漏尿。她正在接受妇科检查和Q-tip测试。问题是关于膀胱测量（cystometry）会显示她的残余尿量和逼尿肌收缩情况。

首先，我知道在像打喷嚏或咳嗽等活动中发生不自主尿液流失通常与压力性尿失禁有关。压力性尿失禁通常发生在尿道肌肉不足以在压力增大的情况下（比如咳嗽时）防止膀胱漏尿时。

接下来是Q-tip测试。根据我记得的，Q-tip是一种用于测量尿道压力曲线的尿道导管。它通常用于评估尿道功能。Q-tip测试阳性结果，即在Valsalva操作过程中尿道压力低于膀胱内压，与内源性括约肌缺陷相关，这是一种压力性尿失禁类型。

由于患者有在活动中出现不自主漏尿的病史，但夜间没有漏尿，更可能是压力性尿失禁，而不是像急迫性尿失禁那样的情况，急迫性尿失禁通常伴有夜间漏尿。因此，如果Q-tip测试阳性，提示内源性括约肌缺陷。

现在，谈到膀胱测量。膀胱测量是一种测试，旨在测量膀胱在充盈过程中的反应以及逼尿肌的收缩情况。它可以显示是否存在膀胱过度活动症（OAB），即引起急迫感和频尿的情况，或是否存在逼尿肌低活动性，导致尿潴留。

在这种情况下，患者的主要问题是压力性尿失禁，这更与无法在压力增大时保持尿液有关。膀胱测量会查看逼尿肌的收缩情况。如果逼尿肌低活动性，它将不能强有力地收缩以排空膀胱，导致残余尿量。但如果逼尿肌过度活跃，可能会收缩过度，导致急迫感。

鉴于患者有压力性尿失禁的病史和Q-tip测试阳性，提示内源性括约肌缺陷，我认为膀胱测量会显示逼尿肌的收缩是正常的。问题不在于逼尿肌收缩的能力，而是无法密封尿道以保持压力。因此，残余尿量可能是正常的，除非有明显的尿潴留，但关键发现是逼尿肌的收缩是正常的，而不是过度活跃。

等等，但会不会有残余尿量？如果患者排尿后膀胱中残留一些尿液，那就是残余尿量。但如果没有尿潴留的症状，比如膀胱饱胀或排尿困难，那么这种情况的可能性较小。主要问题是在活动中发生的尿失禁，因此逼尿肌收缩是正常的，残余尿量在正常范围内，除非有其他情况。

所以，综合来看，膀胱测量可能会显示逼尿肌的收缩正常，残余尿量正常。问题更多是在括约肌方面，而不是逼尿肌。  
\</think>

根据对患者病史和Q-tip测试结果的分析，膀胱测量最可能显示逼尿肌的收缩正常，残余尿量正常。主要问题似乎是由于内源性括约肌缺陷引起的压力性尿失禁，如Q-tip测试阳性所示。这种情况通常影响尿道括约肌在压力增大时防止漏尿的能力，而不是逼尿肌的收缩能力。因此，逼尿肌的收缩并未过度活跃，残余尿量在正常范围内。

标准答案：

在这种压力性尿失禁的情况下，膀胱测压检查（cystometry）最可能显示**正常的排尿后残余尿量**，因为压力性尿失禁通常不会影响膀胱排空功能。此外，由于压力性尿失禁主要与**身体用力**有关，而不是膀胱过度活动症（OAB），因此在测试过程中**不太可能观察到逼尿肌的非自主收缩**。

In [74]:
inputs2 = tokenizer([prompt_style.format(question_2, "")], return_tensors="pt").to("cuda")


outputs2 = model.generate(
    input_ids=inputs2.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

response2 = tokenizer.batch_decode(outputs2)

In [75]:
print(response2[0].split("### Response:")[1])


<think>
Okay, so I have a patient presenting with sudden-onset chest pain that's radiating to the neck and left arm. That makes me think of a heart attack because those symptoms are classic—angina or myocardial infarction. The left arm pain, neck pain, and sometimes jaw or back pain can all be related to coronary artery issues. 

Looking at the past medical history, the patient has hypercholesterolemia, which is high cholesterol, and coronary artery disease. Those are both risk factors for atherosclerosis, which can lead to blockages in the coronary arteries. The elevated troponin I levels are a big clue because troponin is a cardiac enzyme released when the heart muscle is damaged, which is a sign of a heart attack. Also, the patient is experiencing tachycardia, which means their heart is beating faster than usual. In a heart attack, the heart might beat faster as it tries to pump blood to compensate for the blocked artery.

Now, considering the coronary arteries, the left main coron

翻译如下：

\<think>  
好的，我有一位患者，突然出现胸痛，并放射到颈部和左臂。这让我想到了心脏病发作，因为这些症状很经典——心绞痛或心肌梗死。左臂痛、颈部痛，有时还会伴随下颌或背部的疼痛，这些都可能与冠状动脉问题相关。

从病史来看，患者有**高胆固醇血症**（即高胆固醇）和**冠状动脉疾病**，这两个因素都是**动脉粥样硬化**的风险因素，可能导致冠状动脉发生堵塞。**肌钙蛋白I升高**是一个很大的线索，因为肌钙蛋白是心肌受损时释放的心脏酶，通常表明发生了心肌梗死。另外，患者还出现了**心动过速**，即心跳比平常快。在心肌梗死时，心脏可能会加速跳动，以试图通过增加心脏输出量来补偿被阻塞的冠状动脉。

考虑到冠状动脉，**左主冠状动脉**（LMCA）为整个左侧心脏提供血液，包括左心室，而左心室是一个关键的泵血肌肉。如果这里发生堵塞，可能导致更严重的心肌梗死，因为左心室至关重要。**右冠状动脉**为右心室和左心室下壁提供血液，这里的堵塞也是可能的，但**左主冠状动脉**通常与上述症状更相关，尤其是当肌钙蛋白升高时。

所以，将所有因素综合考虑，最可能受累的冠状动脉是**左主冠状动脉**（LMCA）。患者的病史、肌钙蛋白升高以及典型的胸痛放射症状都指向了这一动脉作为罪魁祸首。

\</think>

最可能受累的冠状动脉是**左主冠状动脉（LMCA）**。

**解释：**
- **症状：** 患者突发胸痛并放射至颈部和左臂，以及肌钙蛋白升高，提示急性冠状动脉综合症，可能是心肌梗死。
- **病史：** 高胆固醇血症和冠状动脉疾病病史是动脉粥样硬化的风险因素，可能导致冠状动脉堵塞。
- **心动过速：** 心率增加可能是心脏为补偿心肌血流减少而产生的反应。
- **冠状动脉考虑：** 左主冠状动脉供应左心室，这个肌肉对心脏功能至关重要。与右冠状动脉相比，左主冠状动脉的堵塞会导致更严重且危及生命的心肌梗死，右冠状动脉通常供应的是不那么关键的区域。

因此，症状、肌钙蛋白升高以及患者的病史强烈指向**左主冠状动脉**（LMCA）作为最可能的罪魁祸首。<｜end▁of▁sentence｜>

标准答案：

根据患者表现出的突然胸痛并放射至颈部和左臂，结合其有高胆固醇血症和冠状动脉疾病的病史，肌钙蛋白升高和心动过速，临床症状强烈提示左前降支（LAD）动脉受累。该动脉通常是引发此类症状的罪魁祸首，因为它供应了心脏的大部分区域。放射性疼痛和肌钙蛋白升高的组合表明心肌受损，这使得LAD成为最可能的致病动脉。然而，在没有进一步的诊断检查（如心电图）的情况下，最终的确诊仍需等待确认。

能够看出，在原始状态下，模型能够进行推理并给出回复，但实际上第一个回答过程并不符合医学规范，而第二个问题则直接回答错误。由此可见，在初始状态下，模型对于medical-o1-reasoning-SFT数据集问答效果并不好。

&emsp;&emsp;接下来尝试进行微调，并测试微调后模型问答效果。

### 二、最小可行性实验

&emsp;&emsp;接下来我们尝试进行模型微调，对于当前数据集而言，我们可以带入原始数据集的部分数据进行微调，也可以带入全部数据并遍历多次进行微调。对于大多数的微调实验，我们都可以从最小可行性实验入手进行微调，也就是先尝试带入少量数据进行微调，并观测微调效果。若微调可以顺利执行，并能够获得微调效果，再考虑带入更多的数据进行更大规模微调。

#### 1.数据集准备

&emsp;&emsp;这里我们直接从huggingface上下载medical-o1-reasoning-SFT数据集。

- 设置代理环境

&emsp;&emsp;由于huggingface网络受限，下载数据集前需要先进行网络环境设置。若是AutoDL服务器，则可以按照如下方式开启学术加速，从而顺利连接huggingface并进行数据集下载：

In [84]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

- 下载数据集

&emsp;&emsp;接下来使用datasets进行数据集下载

In [18]:
# !pip install datasets

In [5]:
import os
from datasets import load_dataset


再次确认提示词模板：

In [6]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

然后提取并设置文本生成结束的标记：

In [7]:
EOS_TOKEN = tokenizer.eos_token
tokenizer.eos_token

'<｜end▁of▁sentence｜>'

然后定义函数，用于对medical-o1-reasoning-SFT数据集进行修改，Complex_CoT列和Response列进行拼接，并加上文本结束标记：

In [8]:
def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206180316919.png" alt="image-20250206180316919" style="zoom:50%;" />

在最小可行性实验中，我们可以只下载500条数据进行微调即可看出效果：

In [9]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","zh", split = "train[0:500]",trust_remote_code=True)

In [10]:
dataset[0]

{'Question': '根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？',
 'Complex_CoT': '这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。',
 'Response': '从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿

然后进行结构化处理：

In [11]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

将数据集整理为如下形式：

In [12]:
dataset["text"][0]

'Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本

- 数据集保存地址

默认情况下数据集保存在主目录下.cache文件夹中，数据文件格式如下所示：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206195216257.png" alt="image-20250206195216257" style="zoom:50%;" />

#### 2.开启微调

&emsp;&emsp;然后即可把模型设置为微调模式：

In [13]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  
    loftq_config=None,
)

Unsloth 2025.2.12 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


然后导入相关的库：

In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

创建有监督微调对象：

In [15]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        max_steps=10,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),)

E:\AI\LLM\unsloth_compiled_cache\UnslothSFTTrainer.py:593: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
E:\AI\LLM\unsloth_compiled_cache\UnslothSFTTrainer.py:607: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
E:\AI\LLM\unsloth_compiled_cache\UnslothSFTTrainer.py:631: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
E:\AI\LLM\unsloth_compiled_cache\UnslothSFTTrainer.py:726: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `UnslothSFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


这段代码主要是用 **`SFTTrainer`** 进行 **监督微调（Supervised Fine-Tuning, SFT）**，适用于 `transformers` 和 `Unsloth` 生态中的模型微调：
**1. 导入相关库**
- **`SFTTrainer`**（来自 `trl` 库）：  
  - `trl`（Transformer Reinforcement Learning）是 Hugging Face 旗下的 `trl` 库，提供 **监督微调（SFT）** 和 **强化学习（RLHF）** 相关的功能。
  - `SFTTrainer` 主要用于 **有监督微调（Supervised Fine-Tuning）**，适用于 `LoRA` 等低秩适配微调方式。

- **`TrainingArguments`**（来自 `transformers` 库）：  
  - 这个类用于定义 **训练超参数**，比如批量大小、学习率、优化器、训练步数等。

- **`is_bfloat16_supported()`**（来自 `unsloth`）：  
  - 这个函数检查 **当前 GPU 是否支持 `bfloat16`（BF16）**，如果支持，则返回 `True`，否则返回 `False`。
  - `bfloat16` 是一种更高效的数值格式，在 **新款 NVIDIA A100/H100** 等 GPU 上表现更优。

**2. 初始化 `SFTTrainer` 进行模型微调**

##### **参数解析**
##### **① `SFTTrainer` 部分**
| 参数 | 作用 |
|------|------|
| `model=model` | 指定需要进行微调的 **预训练模型** |
| `tokenizer=tokenizer` | 指定 **分词器**，用于处理文本数据 |
| `train_dataset=dataset` | 传入 **训练数据集** |
| `dataset_text_field="text"` | 指定数据集中哪一列包含 **训练文本**（在 `formatting_prompts_func` 里处理） |
| `max_seq_length=max_seq_length` | **最大序列长度**，控制输入文本的最大 Token 数量 |
| `dataset_num_proc=2` | **数据加载的并行进程数**，提高数据预处理效率 |

##### **② `TrainingArguments` 部分**
| 参数 | 作用 |
|------|------|
| `per_device_train_batch_size=2` | 每个 **GPU/设备** 的训练批量大小（较小值适合大模型） |
| `gradient_accumulation_steps=4` | **梯度累积步数**（相当于 `batch_size=2 × 4 = 8`） |
| `warmup_steps=5` | **预热步数**（初始阶段学习率较低，然后逐步升高） |
| `max_steps=60` | **最大训练步数**（控制训练的总步数，此处总共约消耗60*8=480条数据） |
| `learning_rate=2e-4` | **学习率**（`2e-4` = 0.0002，控制权重更新幅度） |
| `fp16=not is_bfloat16_supported()` | 如果 **GPU 不支持 `bfloat16`，则使用 `fp16`（16位浮点数）** |
| `bf16=is_bfloat16_supported()` | 如果 **GPU 支持 `bfloat16`，则启用 `bfloat16`（训练更稳定）** |
| `logging_steps=10` | **每 10 步记录一次训练日志** |
| `optim="adamw_8bit"` | **使用 `adamw_8bit`（8-bit AdamW优化器）减少显存占用** |
| `weight_decay=0.01` | **权重衰减（L2 正则化）**，防止过拟合 |
| `lr_scheduler_type="linear"` | **学习率调度策略**（线性衰减） |
| `seed=3407` | **随机种子**（保证实验结果可复现） |
| `output_dir="outputs"` | **训练结果的输出目录** |

然后设置wandb（可选）：

In [16]:
import wandb

wandb.login(key="3bedd8f0e8248cda207ca305567e02256f493396")
run = wandb.init(
    project='my fine-tuning on deepseek r1 with medical data',
    job_type="training",
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\kodi\_netrc
wandb: Currently logged in as: 819343713 (819343713-homes-com) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


然后开始微调：

In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 40,370,176
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,10.379800
2,8.826900
3,8.881500
4,8.621000
5,9.779000
6,8.007800
7,6.954200
8,7.213900
9,7.108700
10,6.249200


此时wandb中显示内容如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206200441907.png" alt="image-20250206200441907" style="zoom:50%;" />

In [18]:
trainer_stats

TrainOutput(global_step=10, training_loss=8.202198266983032, metrics={'train_runtime': 3326.9277, 'train_samples_per_second': 0.024, 'train_steps_per_second': 0.003, 'total_flos': 2595710206580736.0, 'train_loss': 8.202198266983032, 'epoch': 0.16})

注意，unsloth在微调结束后，会自动更新模型权重（在缓存中），因此无需手动合并模型权重即可直接调用微调后的模型：

In [19]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(

In [23]:
inputs = tokenizer([prompt_style.format(question_1, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)

In [24]:
print(response[0].split("### Response:")[1])


<think>

The response is based on clinical reasoning, evaluating the findings from the gynecological examination. The gynecologist's report and findings are used to assess the patient's situation. The patient's history, current condition, and results from the examination are considered. The response addresses the evaluation of the findings and their implications for the patient's overall health.

The response explains the clinical reasoning, the evaluation of the findings, and their implications for the patient's condition. It also considers the patient's history and current status to provide a comprehensive assessment of their overall health.

The response is structured to guide the reader through the clinical evaluation, the implications of the findings, and their relevance to the patient's condition. It emphasizes the importance of the findings in determining the patient's overall health and their potential impact on future health.

The conclusion is that the findings from the gyne

测试第二个问题问答效果：

In [108]:
inputs = tokenizer([prompt_style.format(question_2, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)

In [109]:
print(response[0].split("### Response:")[1])


<think>
Alright, let's think about this. We've got a patient with sudden chest pain, and it's radiating to the neck and left arm. That sounds like angina pectoris. Angina is usually due to reduced blood flow to the heart muscle. Now, I remember that there are different types of angina, and each one relates to a specific coronary artery.

Let's consider what we know. The patient has hypercholesterolemia, which means there's a higher level of cholesterol in their blood. This could lead to atherosclerosis, narrowing the arteries over time. They also have coronary artery disease, which means they have some narrowing in their arteries. So, this makes it more likely that they have significant blockages.

Looking at their symptoms, the pain is radiating to the neck and left arm. This sounds a lot like angina due to atherosclerosis of the left main coronary artery, which supplies the entire left ventricle. This artery is crucial because if it's blocked, it can lead to severe issues like a hea

> 此时模型认为“左主冠状动脉最可能是该患者症状的罪魁祸首”，但实际上应该是“左前降支（LAD）动脉受累”导致。

能够发现，第一个问题回答更加规范，并且回答正确。但第二个问题仍然回答错误。由此可以考虑继续进行大规模微调。不过在此之前，我们可以将现在小规模微调的模型进行本地保存。

#### 3.模型合并

此时本地保存的模型权重在`outputs`文件夹中：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206195427494.png" alt="image-20250206195427494" style="zoom:50%;" />

然后可使用如下代码进行模型权重合并：

In [20]:
new_model_local = "DeepSeek-R1-Medical-COT-Tiny"
model.save_pretrained(new_model_local) 
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 7.75 out of 31.82 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


  0%|          | 0/28 [00:00<?, ?it/s]
We will save to Disk and not RAM now.
100%|██████████| 28/28 [00:26<00:00,  1.04it/s]


Unsloth: Saving tokenizer... Done.
Done.


保存结束后，即可在当前文件夹中看到对应模型：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206200056475.png" alt="image-20250206200056475" style="zoom:50%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206200157110.png" alt="image-20250206200157110" style="zoom:50%;" />

然后即可将其推送到huggingface上并保存为GGUF格式文件并进行调用。

In [23]:
# model.save_pretrained_gguf("dir", tokenizer, quantization_method = "q4_k_m")
# model.save_pretrained_gguf("dir", tokenizer, quantization_method = "q8_0")
model.save_pretrained_gguf(r"E:\AI\LLM\DeepSeek-R1-Medical-COT-Tiny", tokenizer, quantization_method = "f16")

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 9.26 out of 31.82 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:39<00:00,  1.42s/it]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...


RuntimeError: Unsloth: The file 'llama.cpp/llama-quantize' or 'llama.cpp/quantize' does not exist.
But we expect this file to exist! Maybe the llama.cpp developers changed the name?

### 三、完整高效微调实验

&emsp;&emsp;接下来我们尝试带入全部数据进行高效微调，以提升模型微调效果。

In [18]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [19]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

此时读取全部数据

In [21]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train",trust_remote_code=True)
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

Map:   0%|          | 0/25371 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nPlease answer the following medical question. \n\n### Question:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Response:\n<think>\nOkay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her ab

In [22]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  
    loftq_config=None,
)

Unsloth 2025.1.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


这里设置epoch为3，遍历3次数据集：

In [26]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs = 3,
        warmup_steps=5,
        # max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/25371 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 25,371 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 9,513
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,1.285900
20,1.262500
30,1.245400
40,1.284500
50,1.319800
60,1.298900
70,1.298400
80,1.251900
90,1.295300
100,1.244500


这里总共训练约15个小时。

In [29]:
trainer_stats

TrainOutput(global_step=9513, training_loss=1.0824475168592858, metrics={'train_runtime': 20193.217, 'train_samples_per_second': 3.769, 'train_steps_per_second': 0.471, 'total_flos': 2.7936033274397737e+18, 'train_loss': 1.0824475168592858, 'epoch': 2.9992117294655527})

带入两个问题进行测试，均有较好的回答效果：

In [30]:
question = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Alright, let's think this through step by step. We've got a 61-year-old woman who's been dealing with involuntary urine loss whenever she does something like coughing or sneezing. That sounds like stress urinary incontinence, which usually means there's some kind of leakage when there's increased pressure in the abdomen. Now, the fact that she doesn't have any leakage at night is interesting. That's a big hint because it suggests that the problem isn't about bladder control or retention, since there's no issue when lying down.

Now, let's consider the Q-tip test. This test is a clever way to check for urethral mobility. If the Q-tip moves a lot, it means the urethra is mobile, which is a typical sign of stress incontinence. So, we're probably looking at a condition where the urethra isn't staying closed under pressure.

Now, if we were to do a cystometry on her, we'd be checking a few things. First, there's the residual volume. Since she's not leaking urine at night, her bladd

In [32]:
question = "Given a patient who experiences sudden-onset chest pain radiating to the neck and left arm, with a past medical history of hypercholesterolemia and coronary artery disease, elevated troponin I levels, and tachycardia, what is the most likely coronary artery involved based on this presentation?"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, let's think about this. The patient is having sudden chest pain that's shooting to their neck and left arm. That's pretty classic for something like a myocardial infarction, right? I mean, the pain spreading like that usually points to a big problem in the heart's supply.

Now, considering the heart's coronary arteries, I know the left anterior descending artery, or LAD, is a big player. It's like the main highway for blood to reach the front wall of the heart. If there's a blockage there, it can definitely cause pain that radiates to the neck and arm.

Then, there's the right coronary artery, or RCA, which supplies the right side of the heart and can affect the inferior wall of the heart. But wait, the pain pattern here seems to be more on the left side, so maybe the RCA is less likely.

The patient has hypercholesterolemia and coronary artery disease. These conditions put them at risk for atherosclerosis, which can lead to blockages in the coronary arteries. The LAD is

最后进行模型权重保存：

In [31]:
new_model_local = "DeepSeek-R1-Medical-COT"
model.save_pretrained(new_model_local) 
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 712.42 out of 1007.51 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 57.91it/s]


Unsloth: Saving tokenizer... Done.
Done.
